In [1]:
import pandas as pd
import os
import psycopg2


#path = 'C:/GitHub/Proyecto_Final_BI/'
path = ""

Lectura de archivos

In [2]:
# función dada en clase
def load_data(filename):
    """ Load dataset from filename
    Args:
        filename (string): Path to data
        
    Returns:
        df (DataFrame): Dataset from file
    """
    df = pd.read_csv(filename)
    return df

filename = os.path.join(path,'Benefits_Cost_Sharing_PUF.csv')
Benefits_CostSharing = load_data(filename)

filename = os.path.join(path,'Business_Rules_PUF.csv')
Business_Rules = load_data(filename)

filename = os.path.join(path,'Plan_Attributes_PUF.csv')
Plan_Attributes = load_data(filename)

filename = os.path.join(path,'Rate_PUF.csv')
Rate = load_data(filename)

/var/folders/nc/msnr18q57_bfryvsdz5vb_w00000gn/T/ipykernel_2861/278177419.py:10: DtypeWarning: Columns (9,12,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
/var/folders/nc/msnr18q57_bfryvsdz5vb_w00000gn/T/ipykernel_2861/278177419.py:10: DtypeWarning: Columns (27,64,80,81,82,86,87,88,89,90,91,111,122,123,124,125,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [3]:
Benefits_CostSharing = Benefits_CostSharing[['StateCode', 'IssuerId', 'SourceName', 'StandardComponentId', 'PlanId', 'BenefitName', 
                                            'CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet', 'CoinsInnTier1', 'CoinsInnTier2',
                                            'CoinsOutofNet', 'IsEHB', 'IsCovered', 'QuantLimitOnSvc', 'LimitQty','LimitUnit', 'Exclusions']]
Benefits_CostSharing.columns

Index(['StateCode', 'IssuerId', 'SourceName', 'StandardComponentId', 'PlanId',
       'BenefitName', 'CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet',
       'CoinsInnTier1', 'CoinsInnTier2', 'CoinsOutofNet', 'IsEHB', 'IsCovered',
       'QuantLimitOnSvc', 'LimitQty', 'LimitUnit', 'Exclusions'],
      dtype='object')

In [5]:
Business_Rules = Business_Rules[['StateCode', 'IssuerId', 'SourceName', 'MedicalDentalBoth', 'StandardComponentId',
                                 'MaximumUnderageDependent','DependentMaximumAgRule','CohabitationRule', 'MarketCoverage',
                                 'DentalOnlyPlan']]
Business_Rules.columns

Index(['StateCode', 'IssuerId', 'SourceName', 'MedicalDentalBoth',
       'StandardComponentId', 'MaximumUnderageDependent',
       'DependentMaximumAgRule', 'CohabitationRule', 'MarketCoverage',
       'DentalOnlyPlan'],
      dtype='object')

In [6]:
Plan_Attributes = Plan_Attributes[['StateCode', 'IssuerId', 'IssuerMarketPlaceMarketingName', 'SourceName', 
                                   'MarketCoverage', 'StandardComponentId', 'PlanMarketingName', 'IsNewPlan',
                                   'PlanType', 'MetalLevel', 'ChildOnlyOffering', 'DiseaseManagementProgramsOffered',
                                   'EHBPercentTotalPremium', 'PlanEffectiveDate', 'PlanExpirationDate', 'PlanId',
                                   'IssuerActuarialValue', 'IsHSAEligible']]
Plan_Attributes.columns

Index(['StateCode', 'IssuerId', 'IssuerMarketPlaceMarketingName', 'SourceName',
       'MarketCoverage', 'StandardComponentId', 'PlanMarketingName',
       'IsNewPlan', 'PlanType', 'MetalLevel', 'ChildOnlyOffering',
       'DiseaseManagementProgramsOffered', 'EHBPercentTotalPremium',
       'PlanEffectiveDate', 'PlanExpirationDate', 'PlanId',
       'IssuerActuarialValue', 'IsHSAEligible'],
      dtype='object')

In [7]:
Rate = Rate[['StateCode', 'IssuerId', 'SourceName','PlanId', 'RatingAreaId',
       'Tobacco', 'Age', 'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents' ]]
Rate.columns

Index(['StateCode', 'IssuerId', 'SourceName', 'PlanId', 'RatingAreaId',
       'Tobacco', 'Age', 'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents'],
      dtype='object')

La tabla de Business Rules PUF puede ser el primer filtro

In [8]:
Business_Rules.head(2)

,StateCode,IssuerId,SourceName,MedicalDentalBoth,StandardComponentId,MaximumUnderageDependent,DependentMaximumAgRule,CohabitationRule,MarketCoverage,DentalOnlyPlan
0,AK,21989,HIOS,Dental,21989AK0030001,3,25,"Child,No;Foster Child,No;Grandson or Granddaug...",Individual,Yes
1,AK,21989,HIOS,Dental,21989AK0030051,3,25,"Child,No;Foster Child,No;Grandson or Granddaug...",SHOP (Small Group),Yes


In [9]:
Business_Rules.StateCode.unique()

array(['AK', 'AL', 'AZ', 'FL', 'GA', 'IN', 'LA', 'MO', 'MS', 'NC', 'OK',
       'TX', 'WI', 'IA', 'ND', 'SC', 'TN', 'WY', 'AR', 'DE', 'HI', 'IL',
       'NE', 'NH', 'OH', 'KS', 'MI', 'MT', 'OR', 'SD', 'UT', 'VA', 'WV'],
      dtype=object)

Virginia: 8642274 VA
Arizona: 7276316 AZ
Tennesse: 6975218 TN


In [12]:
Business_Rules = Business_Rules[(Business_Rules.StateCode.isin(['TX','FL','IL','OH','NC','GA','MI','VA','AZ','TN'])) & (Business_Rules.DentalOnlyPlan == 'No')]

In [13]:
Business_Rules.StateCode.unique()

array(['AZ', 'FL', 'GA', 'NC', 'TX', 'TN', 'IL', 'OH', 'MI', 'VA'],
      dtype=object)

In [14]:
Rate = Rate[Rate.PlanId.isin(Business_Rules.StandardComponentId)]

In [15]:
Plan_Attributes = Plan_Attributes[Plan_Attributes.StandardComponentId.isin(Business_Rules.StandardComponentId)]

In [16]:
Benefits_CostSharing = Benefits_CostSharing[Benefits_CostSharing.StandardComponentId.isin(Business_Rules.StandardComponentId)]

In [17]:
Business_Rules.shape

(3838, 10)

In [18]:
Business_Rules = Business_Rules.drop(columns=['CohabitationRule'])
Business_Rules.head(10)

,StateCode,IssuerId,SourceName,MedicalDentalBoth,StandardComponentId,MaximumUnderageDependent,DependentMaximumAgRule,MarketCoverage,DentalOnlyPlan
135,AZ,13877,HIOS,Medical,13877AZ0070002,3,26,Individual,No
136,AZ,13877,HIOS,Medical,13877AZ0070003,3,26,Individual,No
137,AZ,13877,HIOS,Medical,13877AZ0070005,3,26,Individual,No
138,AZ,13877,HIOS,Medical,13877AZ0070006,3,26,Individual,No
139,AZ,13877,HIOS,Medical,13877AZ0070008,3,26,Individual,No
140,AZ,13877,HIOS,Medical,13877AZ0070011,3,26,Individual,No
141,AZ,13877,HIOS,Medical,13877AZ0070012,3,26,Individual,No
142,AZ,13877,HIOS,Medical,13877AZ0070013,3,26,Individual,No
143,AZ,13877,HIOS,Medical,13877AZ0070014,3,26,Individual,No
144,AZ,13877,HIOS,Medical,13877AZ0070017,3,26,Individual,No


In [20]:
Rate.Age.unique()

array(['0-14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
       '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
       '58', '59', '60', '61', '62', '63', '64 and over'], dtype=object)

In [16]:
state = {'StateCode': ["TX"], 'StateName': ["Texas"]}
state = pd.DataFrame(data=state)
state

,StateCode,StateName
0,TX,Texas


In [17]:
Benefits_CostSharing.columns

Index(['StateCode', 'IssuerId', 'SourceName', 'StandardComponentId', 'PlanId',
       'BenefitName', 'CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet',
       'CoinsInnTier1', 'CoinsInnTier2', 'CoinsOutofNet', 'IsEHB', 'IsCovered',
       'QuantLimitOnSvc', 'LimitQty', 'LimitUnit', 'Exclusions'],
      dtype='object')

In [18]:
Plan_Attributes.columns

Index(['StateCode', 'IssuerId', 'IssuerMarketPlaceMarketingName', 'SourceName',
       'MarketCoverage', 'StandardComponentId', 'PlanMarketingName',
       'IsNewPlan', 'PlanType', 'MetalLevel', 'ChildOnlyOffering',
       'DiseaseManagementProgramsOffered', 'EHBPercentTotalPremium',
       'PlanEffectiveDate', 'PlanExpirationDate', 'PlanId',
       'IssuerActuarialValue', 'IsHSAEligible'],
      dtype='object')

Proceso de imputaciones

In [19]:
Benefits_CostSharing.IsEHB.unique()

array(['Yes', nan], dtype=object)

In [20]:
Benefits_CostSharing = Benefits_CostSharing.fillna("No")

In [21]:
Benefits_CostSharing.IsEHB.unique()

array(['Yes', 'No'], dtype=object)

Insert de tablas en la base de datos

In [22]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd


def execute_values(conn, df, table):

	tuples = [tuple(x) for x in df.to_numpy()]

	cols = ','.join(list(df.columns))
	finish = True
	coma = 0
	cont = 0
	cols_final = '"'
	cols += ","
	while(finish):
		coma = cols.find(",", cont)
		if coma == -1:
			finish = False
			cols_final = cols_final[:-2]
		else:
			var = cols[cont:coma]
			cont = coma + 1
			cols_final += var
			cols_final += '","'


	print(cols_final)
	# SQL query to execute
	query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols_final)
	cursor = conn.cursor()
	try:
		extras.execute_values(cursor, query, tuples)
		conn.commit()
	except (Exception, psycopg2.DatabaseError) as error:
		print("Error: %s" % error)
		conn.rollback()
		cursor.close()
		return 1
	print("execute_values() done")
	cursor.close()

Insert tabla State

In [23]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."States";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = state
# using the function defined
execute_values(conn, data, 'public."States"')
conn.close()

"StateCode","StateName"
execute_values() done


Insert tabla Bussiness Rules

In [32]:
Business_Rules_prueba = Business_Rules[["StateCode", "IssuerId", "MedicalDentalBoth","StandardComponentId"]]
Business_Rules = Business_Rules.drop(columns=['SourceName'])

In [34]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."businessRules";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Business_Rules
# using the function defined
execute_values(conn, data, 'public."businessRules"')
conn.close()

"StateCode","IssuerId","MedicalDentalBoth","StandardComponentId","MaximumUnderageDependent","DependentMaximumAgRule","MarketCoverage","DentalOnlyPlan"
execute_values() done


Insert tabla Benefits_CostSharing

In [36]:
Benefits_CostSharing_prueba = Benefits_CostSharing[["StateCode", "IssuerId", "StandardComponentId", "PlanId"]]
Benefits_CostSharing = Benefits_CostSharing.drop(columns=['SourceName'])

In [38]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."benefitsCostSharing";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Benefits_CostSharing
# using the function defined
execute_values(conn, data, 'public."benefitsCostSharing"')
conn.close()

"StateCode","IssuerId","StandardComponentId","PlanId","BenefitName","CopayInnTier1","CopayInnTier2","CopayOutofNet","CoinsInnTier1","CoinsInnTier2","CoinsOutofNet","IsEHB","IsCovered","QuantLimitOnSvc","LimitQty","LimitUnit","Exclusions"
execute_values() done


Insert tabla Rate

In [43]:
Rate.columns

Index(['StateCode', 'IssuerId', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents'],
      dtype='object')

In [42]:
Rate.dtypes

StateCode                                     object
IssuerId                                       int64
PlanId                                        object
RatingAreaId                                  object
Tobacco                                       object
Age                                           object
IndividualRate                               float64
IndividualTobaccoRate                        float64
Couple                                       float64
PrimarySubscriberAndOneDependent             float64
PrimarySubscriberAndTwoDependents            float64
PrimarySubscriberAndThreeOrMoreDependents    float64
CoupleAndOneDependent                        float64
CoupleAndTwoDependents                       float64
CoupleAndThreeOrMoreDependents               float64
dtype: object

In [40]:
Rate_prueba = Rate[["StateCode", "IssuerId", "PlanId"]]
Rate = Rate.drop(columns=['SourceName'])

In [53]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."Rate";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Rate
# using the function defined
execute_values(conn, data, 'public."Rate"')
conn.close()

"StateCode","IssuerId","PlanId","RatingAreaId","Tobacco","Age","IndividualRate","IndividualTobaccoRate","Couple","PrimarySubscriberAndOneDependent","PrimarySubscriberAndTwoDependents","PrimarySubscriberAndThreeOrMoreDependents","CoupleAndOneDependent","CoupleAndTwoDependents","CoupleAndThreeOrMoreDependents"
execute_values() done


Insert tabla Plan Attributes

In [48]:
Plan_Attributes.columns

Index(['StateCode', 'IssuerId', 'IssuerMarketPlaceMarketingName',
       'MarketCoverage', 'StandardComponentId', 'PlanMarketingName',
       'IsNewPlan', 'PlanType', 'MetalLevel', 'ChildOnlyOffering',
       'DiseaseManagementProgramsOffered', 'EHBPercentTotalPremium',
       'PlanEffectiveDate', 'PlanExpirationDate', 'PlanId',
       'IssuerActuarialValue', 'IsHSAEligible'],
      dtype='object')

In [49]:
Plan_Attributes.dtypes

StateCode                            object
IssuerId                              int64
IssuerMarketPlaceMarketingName       object
MarketCoverage                       object
StandardComponentId                  object
PlanMarketingName                    object
IsNewPlan                            object
PlanType                             object
MetalLevel                           object
ChildOnlyOffering                    object
DiseaseManagementProgramsOffered     object
EHBPercentTotalPremium              float64
PlanEffectiveDate                    object
PlanExpirationDate                   object
PlanId                               object
IssuerActuarialValue                 object
IsHSAEligible                        object
dtype: object

In [47]:
Plan_Attributes_prueba = Plan_Attributes[["StateCode", "IssuerId", "IssuerMarketPlaceMarketingName", "MarketCoverage", "StandardComponentId"]]
Plan_Attributes = Plan_Attributes.drop(columns = ['SourceName'])

In [52]:
# establishing connection
conn = psycopg2.connect(
  database="proyectoBI",
  user='postgres', 
  password='admin', 
  host='localhost', 
  port= '5432'
)
sql = '''DELETE FROM public."planAttributes";'''
  
#creating a cursor
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
data = Plan_Attributes
# using the function defined
execute_values(conn, data, 'public."planAttributes"')
conn.close()

"StateCode","IssuerId","IssuerMarketPlaceMarketingName","MarketCoverage","StandardComponentId","PlanMarketingName","IsNewPlan","PlanType","MetalLevel","ChildOnlyOffering","DiseaseManagementProgramsOffered","EHBPercentTotalPremium","PlanEffectiveDate","PlanExpirationDate","PlanId","IssuerActuarialValue","IsHSAEligible"
execute_values() done
